In [3]:
import matplotlib.pyplot as plt
import PIL
import numpy as np
import os
import shutil
import cv2
from numpy.random import randint
import pandas as pd 
import scipy.io as sc
import h5py

### We have 17 classes in our dataset

In [4]:
correct_classes = {'none_none_none_none': 0,
                   'none_walk_none_none': 1,
                   'none_stand_reach_top': 2,
                   'box_stand_pick-up_top': 3,
                   'box_stand_place_mid': 4,   
                   'none_stand_none_none': 5,                
                   'rod_stand_pick-up_top': 6,
                   'rod_stand_place_mid': 7,
                   'box_stand_pick-up_mid': 8,
                   'rod_stand_pick-up_mid': 9,
                   'none_bend_none_none': 10,
                   'rod_bend_pick-up_low': 11,
                   'box_bend_place_low': 12,
                   'rod_bend_place_low': 13,                   
                   'box_stand_place_top': 14,
                   'rod_stand_place_top': 15,
                   'box_bend_pick-up_low': 16,
                   'box_walk_hold_none': 17}

In [5]:
dataDir='./UW_IOM_Dataset/'

In [6]:
# Labels format for UW_dataset:
# labels=[mid_level, high_level, low_level, ergonomic_risk]

low_level=['none','top','mid','low']
mid_level_L=['none','walk', 'bend', 'stand'] #, 'twist'
mid_level_H=['none','pick-up','place','reach','hold']
high_level=['none','box', 'rod']

ergonomic_risk = ['none','low-risk','medium-risk','high-risk']


In [16]:
def ExtractFrame(dataDir):    
    video_path =os.path.join(dataDir,'Videos')
#     print(video_path)
    for i, file in enumerate(os.listdir(video_path)):
        file_dir = os.path.join(video_path, file)
#         print(file_dir)
        filename = file[:-4]
        print("++++++++",filename,"++++++++")
        cap = cv2.VideoCapture(file_dir)
        # Check if camera opened successfully
        if (cap.isOpened()== False): 
            print("Error opening video stream or folder")
        labels = read_labels(filename+'.txt', dataDir)
#         
        Frames=[]; count=-1; #training_labels=[]
        label_count = 0;
        # Read until video is completed
        while(cap.isOpened()):
            # Capture frame-by-frame
            count += 1
            ret, frame = cap.read()
            #if ret == True and Trim_info[i][0]<count<Trim_info[i][1]:
            if ret == True and count<len(labels):
                # change the folder name for different frame extracion
                training_frame_folder_path = os.path.join('./', 'Frames',str(filename)) #or Frames_original
                #We make the folder path.
                if not os.path.isdir(training_frame_folder_path):
                    os.makedirs(training_frame_folder_path)
                
                training_frame_path = training_frame_folder_path+"/%d.jpg" % label_count
#                 *******************************************************************
                frame=cv2.resize(frame, (224, 224))  # resize the frames
                cv2.imwrite(training_frame_path, frame)     # save frame as JPEG file
                
                label_count += 1
                # Press Q on keyboard to  exit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            #elif count==Trim_info[i][1]:
            else:      
                break
            
        # When everything done, release the video capture object
        cap.release()
        # Closes all the frames
        cv2.destroyAllWindows()

    return 

def read_labels(filename, dataDir):
    labels_path =os.path.join(dataDir,'VideoLabels')
    with open(os.path.join(labels_path, filename), 'r') as f:
        x = f.readlines()
    labels=[]
    for i, x_i  in enumerate(x):
        if i>0:
            labels.append([int(x) for x in x_i.split('\t')[2:-1]][1::2])
    return labels

def ExtractFrame_in_classs(dataDir):
    video_path = os.path.join(dataDir,'Videos')
    labels_tag_path = os.path.join(dataDir,'VideoLabels/')
    labels_num_path = os.path.join(dataDir,'VideoLabelssNum/')
 
    for i, file in enumerate(os.listdir(video_path)):
        
        filename = file[:-4]
        file_dir = os.path.join(video_path, file)

        print("++++++++",filename,"++++++++")
        
        cap = cv2.VideoCapture(file_dir)
        # Check if camera opened successfully
        if (cap.isOpened()== False): 
            print("Error opening video stream or folder")

        labelfile = open(labels_tag_path+filename+'.txt', 'r')
        labels = [line[:-1] for line in labelfile.readlines()]

        Frames=[]; count=-1; #training_labels=[]
        # Read until video is completed
        label_count = 0;
        while(cap.isOpened()):
            # Capture frame-by-frame
            count += 1
            # Capture frame-by-frame
            ret, frame = cap.read()
            
            #if ret == True  and Trim_info[i][0]<count<Trim_info[i][1]:
            if ret == True  and count<len(labels):
                training_frame_folder_path = os.path.join('./', 'Training_frames_each_video_with_classes',filename,labels[label_count])
                #We make the folder path.
                if not os.path.isdir(training_frame_folder_path):
                    os.makedirs(training_frame_folder_path)
                
                training_frame_path = training_frame_folder_path+"/%d_" % label_count + labels[label_count]+'_'+ str(i+1)+".jpg"
                frame=cv2.resize(frame, (224, 224))
                
                
                cv2.imwrite(training_frame_path, frame)
                    # save frame as JPEG file

                label_count += 1
                # Press Q on keyboard to  exit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            #elif count==Trim_info[i][1]:
            else:
                break
        # When everything done, release the video capture object
        cap.release()
        # Closes all the frames
        cv2.destroyAllWindows()
    return 

def all_frames_in_classes():
#     print(img_root_path)
    img_root_path = './Training_frames_each_video_with_classes'
    saving_path = './Training_frames_All_classes/'
    for i, folder in enumerate(os.listdir(img_root_path)):
        print(folder)
        sub_folder_path = os.path.join(img_root_path,folder)
        for j, sub_folder in enumerate(os.listdir(sub_folder_path)):
            print(sub_folder)
            class_path = os.path.join(saving_path,sub_folder) #this is the destination path
            if not os.path.isdir(class_path):
                os.makedirs(class_path)
            source_path =  os.path.join(sub_folder_path,sub_folder) #this is the source path
            for img in os.listdir(source_path):
                shutil.move(os.path.join(source_path,img), class_path)

    return

def make_train_val():
#     print(img_root_path)
    img_root_path = './Training_frames_All_classes'
    print('******** Important: These are not the actual class numbers, they are just counts. Correct class numbers are shown on top of this file. ***************')
    for i, file in enumerate(os.listdir(img_root_path)):
        
        img_path = os.path.join(img_root_path,file)
        train_path = os.path.join('./Train_Valid','train',file)
#         print(train_path)
        if not os.path.isdir(train_path):
                    os.makedirs(train_path)
        valid_path = os.path.join('./Train_Valid','valid',file)
#         print(valid_path)
        if not os.path.isdir(valid_path):
                    os.makedirs(valid_path)
                
        print('class',i+1,': ',file)
        
        data_size = len(os.listdir(img_path))
        val_size = int(.2*data_size)
        val_ind = randint(0,data_size-1,val_size)
        print('data_size:',data_size)
        print('train_size:',data_size-val_size)
        print('val_size:',val_size)
        count=0
        while count<data_size:
#             print(valid_path+'/'+os.listdir(img_path)[0])
            if count in val_ind:
                shutil.move(img_path+'/'+os.listdir(img_path)[0], valid_path+'/'+os.listdir(img_path)[0])
            else:
                shutil.move(img_path+'/'+os.listdir(img_path)[0], train_path+'/'+os.listdir(img_path)[0])
##             os.rename("path/to/current/file.foo", "path/to/new/destination/for/file.foo") does the same but ...
            count+=1
        print('count:', count)

    return

### 1- Extracts frames of every video in the dataset and stores the frames in a folder with the video name. This is going to be used later for feature extraction

In [8]:
ExtractFrame(dataDir)

++++++++ 01 ++++++++
++++++++ 02 ++++++++
++++++++ 03 ++++++++
++++++++ 04 ++++++++
++++++++ 05 ++++++++
++++++++ 06 ++++++++
++++++++ 07 ++++++++
++++++++ 08 ++++++++
++++++++ 09 ++++++++
++++++++ 10 ++++++++
++++++++ 11 ++++++++
++++++++ 12 ++++++++
++++++++ 13 ++++++++
++++++++ 14 ++++++++
++++++++ 15 ++++++++
++++++++ 16 ++++++++
++++++++ 17 ++++++++
++++++++ 18 ++++++++
++++++++ 19 ++++++++
++++++++ 20 ++++++++


### 2- Make a folder for every video with subfolders containing the frames corressponding to a class

In [9]:
ExtractFrame_in_classs(dataDir)

++++++++ 01 ++++++++
++++++++ 02 ++++++++
++++++++ 03 ++++++++
++++++++ 04 ++++++++
++++++++ 05 ++++++++
++++++++ 06 ++++++++
++++++++ 07 ++++++++
++++++++ 08 ++++++++
++++++++ 09 ++++++++
++++++++ 10 ++++++++
++++++++ 11 ++++++++
++++++++ 12 ++++++++
++++++++ 13 ++++++++
++++++++ 14 ++++++++
++++++++ 15 ++++++++
++++++++ 16 ++++++++
++++++++ 17 ++++++++
++++++++ 18 ++++++++
++++++++ 19 ++++++++
++++++++ 20 ++++++++


### 3- Put together all frames corresponding to a class and move them all in one folder containing all classes

In [13]:
all_frames_in_classes()

01
box_bend_pick-up_low
box_bend_place_low
box_stand_pick-up_mid
box_stand_pick-up_top
box_stand_place_mid
box_stand_place_top
box_walk_hold_none
none_bend_none_none
none_stand_none_none
none_stand_reach_top
none_walk_none_none
rod_bend_pick-up_low
rod_bend_place_low
rod_stand_pick-up_mid
rod_stand_pick-up_top
rod_stand_place_mid
rod_stand_place_top
02
box_bend_pick-up_low
box_bend_place_low
box_stand_pick-up_mid
box_stand_pick-up_top
box_stand_place_mid
box_stand_place_top
box_walk_hold_none
none_bend_none_none
none_stand_none_none
none_stand_reach_top
none_walk_none_none
rod_bend_pick-up_low
rod_bend_place_low
rod_stand_pick-up_mid
rod_stand_pick-up_top
rod_stand_place_mid
rod_stand_place_top
03
box_bend_pick-up_low
box_bend_place_low
box_stand_pick-up_mid
box_stand_pick-up_top
box_stand_place_mid
box_stand_place_top
box_walk_hold_none
none_bend_none_none
none_stand_none_none
none_stand_reach_top
none_walk_none_none
rod_bend_pick-up_low
rod_bend_place_low
rod_stand_pick-up_mid
rod_st

### 4- Build training and validation sets for VGG16

In [17]:
make_train_val()

******** Important: These are not the actual class numbers, they are just counts. Correct class numbers are shown on top of this file. ***************
class 1 :  box_bend_pick-up_low
data_size: 924
train_size: 740
val_size: 184
count: 924
class 2 :  box_bend_place_low
data_size: 1031
train_size: 825
val_size: 206
count: 1031
class 3 :  box_stand_pick-up_mid
data_size: 4807
train_size: 3846
val_size: 961
count: 4807
class 4 :  box_stand_pick-up_top
data_size: 917
train_size: 734
val_size: 183
count: 917
class 5 :  box_stand_place_mid
data_size: 4357
train_size: 3486
val_size: 871
count: 4357
class 6 :  box_stand_place_top
data_size: 1311
train_size: 1049
val_size: 262
count: 1311
class 7 :  box_walk_hold_none
data_size: 382
train_size: 306
val_size: 76
count: 382
class 8 :  none_bend_none_none
data_size: 2195
train_size: 1756
val_size: 439
count: 2195
class 9 :  none_stand_none_none
data_size: 2575
train_size: 2060
val_size: 515
count: 2575
class 10 :  none_stand_reach_top
data_size: 12

### 5- Generating random splits for TCN

In [20]:
import random
video_count = 10 # number of videos
split_count = 5 # number of splits
valid_count = 3 # number of validation videos
training_count = video_count-valid_count # number of training videos

A = np.arange(1,video_count+1)

print('Videos: ',A)

for i in range(split_count):
    split_path = os.path.join('./Splits/','Split_'+str(i+1)) #this is the destination path
    print(split_path)
    if not os.path.isdir(split_path):
        os.makedirs(split_path)
    np.savetxt('./Splits/'+'/all.txt',A)
    e = random.sample(range(1, video_count+1), training_count)
#     print(np.shape(e))
    print('Training set '+str(i+1)+': ',e)
    np.savetxt(split_path+'/train.txt',e)
    v = A[np.where(np.isin(A,e)==False)[0]]
#     print(np.shape(v))
    print('Validation set '+str(i+1)+': ',v)
    np.savetxt(split_path+'/test.txt',v)

Videos:  [ 1  2  3  4  5  6  7  8  9 10]
./Splits/Split_1
Training set 1:  [9, 7, 4, 1, 3, 8, 10]
Validation set 1:  [2 5 6]
./Splits/Split_2
Training set 2:  [9, 4, 6, 5, 7, 2, 10]
Validation set 2:  [1 3 8]
./Splits/Split_3
Training set 3:  [5, 7, 6, 10, 1, 2, 3]
Validation set 3:  [4 8 9]
./Splits/Split_4
Training set 4:  [5, 10, 2, 3, 7, 9, 6]
Validation set 4:  [1 4 8]
./Splits/Split_5
Training set 5:  [3, 2, 9, 7, 6, 10, 1]
Validation set 5:  [4 5 8]
